<a href="https://colab.research.google.com/github/ArnoldoOliva/DatosMasivos/blob/main/modelo_30_jun_dm_SEMISUPERVISADO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",None)

In [27]:
b=pd.read_csv("/content/bot_detection_data.csv")
b.head(5)

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention


Modelo semisupervisado

In [28]:
#omitiremos la parte del text analysis:
b=b[["Retweet Count","Mention Count","Follower Count","Verified","Bot Label","Created At"]]
b["Created At"]=pd.to_datetime(b["Created At"],errors="coerce")

#creamos columna de que tan nuevo es
from datetime import datetime

fecha_hoy = datetime.now().date()

b['datecount'] = fecha_hoy - b['Created At'].dt.date
b.head(3)

<ipython-input-28-5e45271936fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b["Created At"]=pd.to_datetime(b["Created At"],errors="coerce")


,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Created At,datecount
0,85,1,2353,False,1,2020-05-11 15:29:50,1146 days
1,55,5,9617,True,0,2022-11-26 05:18:10,217 days
2,6,2,4363,True,0,2022-08-08 03:16:54,327 days


Manipularemos los datos, forzandolos a ser una parte no supervisados

In [29]:
num_rows_to_null = round(len(b)*.2)

# Get random indices of rows to set as null
random_indices = np.random.choice(b.index, size=num_rows_to_null, replace=False)

b["bot label"]=b["Bot Label"]

# Set the selected rows as null values
b.loc[random_indices, 'bot label'] = np.nan

In [30]:
b.datecount=b['datecount'].dt.days.astype(int)
b.drop("Created At",axis=1,inplace=True)
b.head(2)

,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,datecount,bot label
0,85,1,2353,False,1,1146,NaN
1,55,5,9617,True,0,217,0.0


In [31]:
print(b.head(2))

   Retweet Count  Mention Count  Follower Count  Verified  Bot Label  \
0             85              1            2353     False          1   
1             55              5            9617      True          0   

   datecount  bot label  
0       1146        NaN  
1        217        0.0  


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.cluster import KMeans


# Split the data into labeled and unlabeled examples based on Bot Label column
labeled_df = b.dropna(subset=['bot label'])
unlabeled_df = b[b['bot label'].isnull()]

# Select the features and target variable
features = ['Retweet Count', 'Mention Count', 'Follower Count', 'Verified', 'datecount']
target = 'bot label'

# Split the labeled data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(labeled_df[features], labeled_df[target], test_size=0.2, random_state=42)

# Train a supervised model on the labeled data
model = SVC()
model.fit(X_train, y_train)

# Predict the labels for the unlabeled data  (pseudo)
unlabeled_df['bot label'] = model.predict(unlabeled_df[features])



<ipython-input-33-51431c97d862>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlabeled_df['bot label'] = model.predict(unlabeled_df[features])


In [34]:
# Accuracy of the model over the labeled data:

from sklearn.metrics import accuracy_score

# Make predictions on the labeled data
y_test_pred = model.predict(X_test)

# Calculate the accuracy of the SVM model on the labeled data
accuracy = accuracy_score(y_test, y_test_pred)

print("Accuracy of the SVM model on labeled data:", accuracy) #bad accuracy


Accuracy of the SVM model on labeled data: 0.49575


In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_test_pred)

print("Confusion Matrix:")
print(confusion_matrix)        #umbalanced

Confusion Matrix:
[[ 959 3053]
 [ 981 3007]]


In [36]:
unlabeled_df["bot label"].value_counts()

1.0    7547
0.0    2453
Name: bot label, dtype: int64

In [ ]:
# Combine the labeled and pseudo-labeled data
combined_df = pd.concat([labeled_df, unlabeled_df])

# Train a clustering model on the combined data
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(combined_df[features])

# Assign labels to the unlabeled data based on clustering results
unlabeled_df['Bot Label -algorithms'] = kmeans.predict(unlabeled_df[features])

# Combine the labeled and pseudo-labeled data again
final_df = pd.concat([labeled_df, unlabeled_df])

In [ ]:
unlabeled_df

In [52]:
from sklearn.metrics import confusion_matrix

cmnosupervisado=confusion_matrix(unlabeled_df["Bot Label"].values, unlabeled_df["Bot Label -algorithms"].values)
print(cmnosupervisado)
print(accuracy_score(unlabeled_df["Bot Label"].values, unlabeled_df["Bot Label -algorithms"].values))

[[2481 2527]
 [2490 2502]]
0.4983


In [44]:
#predict last set of data with supervised:
svmdf=final_df[final_df["Bot Label -algorithms"].isna()]
svmdf['Bot Label -algorithms'] = model.predict(svmdf[features])
svmdf

<ipython-input-44-43795f05c7c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  svmdf['Bot Label -algorithms'] = model.predict(svmdf[features])


,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,datecount,bot label,Bot Label -algorithms
1,55,5,9617,True,0,217,0.0,1.0
2,6,2,4363,True,0,327,0.0,0.0
3,54,5,2242,True,1,686,1.0,1.0
4,26,3,8438,False,1,1174,1.0,1.0
5,41,4,3792,True,1,55,1.0,0.0
...,...,...,...,...,...,...,...,...
49995,64,0,9911,True,1,72,1.0,1.0
49996,18,5,9900,False,1,256,1.0,1.0
49997,43,3,6313,True,1,1088,1.0,1.0
49998,45,1,6343,False,0,466,0.0,0.0


In [45]:
final_df.loc[final_df["Bot Label -algorithms"].isna(),"Bot Label -algorithms"]=svmdf["Bot Label -algorithms"]

In [48]:
from sklearn.metrics import confusion_matrix

fconfusion_matrix = confusion_matrix(final_df["Bot Label"].values, final_df["Bot Label -algorithms"].values)

print("Confusion Matrix:")
print(fconfusion_matrix)

Confusion Matrix:
[[ 7395 17587]
 [ 7160 17858]]


In [49]:
final_df["Bot Label"].value_counts()

1    25018
0    24982
Name: Bot Label, dtype: int64

In [50]:
# Calculate the accuracy of the SVM model on the labeled data
accuracy = accuracy_score(final_df["Bot Label"].values, final_df["Bot Label -algorithms"].values)

print("Accuracy of the SVM model on labeled data:", accuracy) #bad accuracy

Accuracy of the SVM model on labeled data: 0.50506


El resultado final de combinar ambas metodologías de unsupervised learning con Kmeand y supervised learning con SVM (para realizar la parte de no supervisado se sustituyeron algunos valores reales de la variable Bot Label con nulos), fue muy pobre. En sus respectivas secciones de predicción, ambas metodologías resultaron con accuracies de alrededor del 49%-50%, muy bajo a decir verdad. Obviamente al juntarlas en la base total, el resultado fue el mismo. Esto puede ser debido a que tal vez las variables numéricas por si solas no sirvieron de mucho para las predicciones de ambas técnicas.